In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import io
import requests
from bs4 import BeautifulSoup
import urllib.request
import re
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [24]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, 'html.parser')

In [28]:
data = []
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])
cd = pd.DataFrame(data, columns=["PostalCode", "Borough", "Neighborhood"])
cd.drop(cd.index[0],inplace = True)
cd.tail(10)

,PostalCode,Borough,Neighborhood
279,M4Z,Not assigned,Not assigned
280,M5Z,Not assigned,Not assigned
281,M6Z,Not assigned,Not assigned
282,M7Z,Not assigned,Not assigned
283,M8Z,Etobicoke,Kingsway Park South West
284,M8Z,Etobicoke,Mimico NW
285,M8Z,Etobicoke,The Queensway West
286,M8Z,Etobicoke,Royal York South West
287,M8Z,Etobicoke,South of Bloor
288,M9Z,Not assigned,Not assigned


In [29]:
cd = cd[cd.Borough != 'Not assigned']
cd.shape

(211, 3)

In [30]:
cd = cd.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
cd.shape

(103, 3)

In [33]:
index = cd['Neighborhood'] == 'Not assigned'
cd.loc[index,'Neighborhood'] = cd[index].Borough

In [35]:
url="https://cocl.us/Geospatial_data"
s=requests.get(url).content
data=pd.read_csv(io.StringIO(s.decode('utf-8')))
data.tail(10)

,Postal Code,Latitude,Longitude
93,M9A,43.667856,-79.532242
94,M9B,43.650943,-79.554724
95,M9C,43.643515,-79.577201
96,M9L,43.756303,-79.565963
97,M9M,43.724766,-79.532242
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


In [37]:
cd['Latitude'] = data['Latitude']
cd['Longitude'] = data['Longitude']
cd.tail(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
93,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
94,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724
95,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
96,M9L,North York,Humber Summit,43.756303,-79.565963
97,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
102,M9W,Etobicoke,Northwest,43.706748,-79.594054


In [38]:
Toronto=cd[cd['Borough'].str.contains('Toronto')]
Toronto.tail(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
69,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
70,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
75,M6G,Downtown Toronto,Christie,43.669542,-79.422564
76,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
77,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
78,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
82,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
83,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
84,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450
87,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558


In [39]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [41]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [42]:
# Foursquare API DATA
CLIENT_ID = 'GNXOMGYD1ITNGHOOIMRM5FDVDGWHGPUN42NASMVYMVMLAF1Q' # your Foursquare ID
CLIENT_SECRET = 'TU5ZXSADG4YBXJD5AQFPNQCN5NQ10W5D033HYCEMWJDD44PQ' # your Foursquare Secret
VERSION = '20180323' # Foursquare API version

In [55]:
Toronto.loc[45, 'Neighborhood']

'Davisville North'

In [56]:
nb_latitude = Toronto.loc[45, 'Latitude']
nb_longitude = Toronto.loc[45, 'Longitude']

nb_name = Toronto.loc[45, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(nb_name, 
                                                               nb_latitude, 
                                                               nb_longitude))

Latitude and longitude values of Davisville North are 43.7127511, -79.3901975.


In [57]:
LIMIT = 50 
Radius = 800
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    nb_latitude, 
    nb_longitude, 
    Radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=GNXOMGYD1ITNGHOOIMRM5FDVDGWHGPUN42NASMVYMVMLAF1Q&client_secret=TU5ZXSADG4YBXJD5AQFPNQCN5NQ10W5D033HYCEMWJDD44PQ&v=20180323&ll=43.7127511,-79.3901975&radius=800&limit=50'

In [58]:
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5ccdf03e4434b9319e74565b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Davisville',
  'headerFullLocation': 'Davisville, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 51,
  'suggestedBounds': {'ne': {'lat': 43.7199511072, 'lng': -79.38025501106614},
   'sw': {'lat': 43.70555109279999, 'lng': -79.40013998893386}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ba011c2f964a5204a5737e3',
       'name': 'Sherwood Park',
       'contact': {},
       'location': {'address': '190 Sherwood Ave',
        'lat': 43.71655100307589,
        'lng': -79.38777567141624,
        'labeledLatLngs': [{'label': 'display',
       

In [59]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [62]:
venues = result['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Sherwood Park,Park,43.716551,-79.387776
1,Summerhill Market North,Food & Drink Shop,43.715499,-79.392881
2,Istanbul Cafe & Espresso Bar,Café,43.707891,-79.393049
3,Loblaws,Supermarket,43.707412,-79.394909
4,Homeway Restaurant & Brunch,Breakfast Spot,43.712641,-79.391557
5,Starbucks,Coffee Shop,43.711200,-79.399182
6,De Mello Palheta Coffee Roasters,Coffee Shop,43.711791,-79.399403
7,DAVIDsTEA,Tea Room,43.709765,-79.398941
8,Douce France,Bakery,43.711554,-79.399394
9,La Vecchia Ristorante,Italian Restaurant,43.710167,-79.399086


In [66]:
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
American Restaurant,1,1,1
Bakery,1,1,1
Breakfast Spot,1,1,1
Brewery,1,1,1
Bubble Tea Shop,1,1,1
Burger Joint,1,1,1
Café,3,3,3
Coffee Shop,3,3,3
Convenience Store,1,1,1


In [68]:
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 35 uniques categories.


In [73]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            Radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [75]:
Toronto_Venues = getNearbyVenues(names=Toronto['Neighborhood'],
                                   latitudes=Toronto['Latitude'],
                                   longitudes=Toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [76]:
print(Toronto_Venues.shape)
Toronto_Venues.head()

(1652, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
1,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
2,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
3,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
4,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery


In [78]:
Toronto_Venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"Brockton, Exhibition Place, Parkdale Village",50,50,50,50,50,50
Business Reply Mail Processing Centre 969 Eastern,50,50,50,50,50,50
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",29,29,29,29,29,29
"Cabbagetown, St. James Town",50,50,50,50,50,50
Central Bay Street,50,50,50,50,50,50
"Chinatown, Grange Park, Kensington Market",50,50,50,50,50,50
Christie,50,50,50,50,50,50


In [80]:
print('There are {} uniques categories.'.format(len(Toronto_Venues['Venue Category'].unique())))

There are 239 uniques categories.


In [83]:
Toronto_Val = pd.get_dummies(Toronto_Venues[['Venue Category']], prefix="", prefix_sep="")

Toronto_Val['Neighborhood'] = Toronto_Venues['Neighborhood'] 

fixed_columns = [Toronto_Val.columns[-1]] + list(Toronto_Val.columns[:-1])
Toronto1 = Toronto_Val[fixed_columns]

Toronto1.head(10)

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Noodle House,Office,Opera House,Organic Grocery,Pakistani Restaurant,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [84]:
Toronto1.shape

(1652, 239)

In [85]:
organized = Toronto1.groupby('Neighborhood').mean().reset_index()
organized

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Noodle House,Office,Opera House,Organic Grocery,Pakistani Restaurant,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.040000,0.00,0.00,0.000000,0.000000,0.000000,0.020000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00000,0.00,0.00,0.000000,0.000000,0.000000,0.02000,0.040000,0.00000,0.00,0.02,0.02,0.000000,0.00,0.00,0.080000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.060000,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.020000,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00000,0.00,0.00,0.00,0.00,0.000000,

In [86]:
organized.shape

(38, 239)

In [90]:
num_top_venues = 5

for hood in organized['Neighborhood']:
    print("----"+hood+"----")
    gp = organized[organized['Neighborhood'] == hood].T.reset_index()
    gp.columns = ['venue','freq']
    gp = gp.iloc[1:]
    gp['freq'] = gp['freq'].astype(float)
    gmp = gp.round({'freq': 2})
    print(gp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0  American Restaurant  0.08
1                 Café  0.08
2          Coffee Shop  0.06
3           Steakhouse  0.06
4         Concert Hall  0.04


----Berczy Park----
                 venue  freq
0                 Café  0.06
1          Coffee Shop  0.06
2   Seafood Restaurant  0.04
3               Bakery  0.04
4  Japanese Restaurant  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0            Café  0.08
1     Coffee Shop  0.08
2           Hotel  0.04
3  Sandwich Place  0.04
4             Bar  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0  Fast Food Restaurant  0.08
1                   Bar  0.06
2         Grocery Store  0.04
3         Burrito Place  0.04
4                  Park  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue      freq
0   Harbor 

In [91]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [118]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
NBV_sorted = pd.DataFrame(columns=columns)
NBV_sorted['Neighborhood'] = organized['Neighborhood']

for ind in np.arange(organized.shape[0]):
    NBV_sorted.iloc[ind, 1:] = return_most_common_venues(organized.iloc[ind, :], num_top_venues)

In [119]:
NBV_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,American Restaurant,Steakhouse,Coffee Shop,Pizza Place,Asian Restaurant,Breakfast Spot,Concert Hall,Hotel,Gastropub
1,Berczy Park,Café,Coffee Shop,Creperie,Farmers Market,Japanese Restaurant,Cheese Shop,Seafood Restaurant,Bakery,Cocktail Bar,Cosmetics Shop
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Bakery,Restaurant,Hotel,Gift Shop,Furniture / Home Store,Bar,Sandwich Place,Indian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Fast Food Restaurant,Bar,Park,Pizza Place,Grocery Store,Italian Restaurant,Light Rail Station,Brewery,Burrito Place,Skate Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Harbor / Marina,Boat or Ferry,Airport Service,Airport Terminal,Sculpture Garden,Airport Lounge,Music Venue,Boutique,Rental Car Location,Pier
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Park,Thai Restaurant,Gastropub,Diner,Italian Restaurant,Beer Store,Bakery
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Pizza Place,Japanese Restaurant,Bubble Tea Shop,Tea Room,Yoga Studio,Ramen Restaurant,Bar
7,"Chinatown, Grange Park, Kensington Market",Café,Mexican Restaurant,Vegetarian / Vegan Restaurant,Burger Joint,Coffee Shop,Vietnamese Restaurant,Bakery,Bar,Caribbean Restaurant,Cocktail Bar
8,Christie,Korean Restaurant,Grocery Store,Coffee Shop,Café,Park,Pizza Place,Diner,Japanese Restaurant,Indian Restaurant,Cocktail Bar
9,Church and Wellesley,Coffee Shop,Gastropub,Gay Bar,Japanese Restaurant,Men's Store,Restaurant,Pizza Place,Park,Mexican Restaurant,Pub


In [120]:
kclusters = 10

t_clustering = organized.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(t_clustering)

kmeans.labels_[0:10]

array([1, 4, 6, 0, 9, 6, 8, 4, 6, 6], dtype=int32)

In [121]:
NBV_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto2 = Toronto

Toronto2 = Toronto2.join(NBV_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto2.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Pub,Bar,Breakfast Spot,Sandwich Place,Café,Japanese Restaurant,Bookstore,Asian Restaurant,Gastropub,Bakery
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,4,Greek Restaurant,Coffee Shop,Pub,Bakery,Ice Cream Shop,Italian Restaurant,Café,Yoga Studio,Restaurant,Japanese Restaurant
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Indian Restaurant,Sandwich Place,Gym,Brewery,Fast Food Restaurant,Bus Stop,Café,Coffee Shop,Park,Convenience Store
43,M4M,East Toronto,Studio District,43.659526,-79.340923,8,Coffee Shop,Bakery,Café,Italian Restaurant,Bar,American Restaurant,Rock Club,Boutique,Restaurant,Comfort Food Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,5,Gym / Fitness Center,Bookstore,Park,Café,Restaurant,Bus Line,Coffee Shop,General Entertainment,Design Studio,Dumpling Restaurant


In [122]:
map_cluster = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(Toronto2['Latitude'], Toronto2['Longitude'], Toronto2['Neighborhood'], Toronto2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster)
       
map_cluster

In [124]:
Toronto2.loc[Toronto2['Cluster Labels'] == 0, Toronto2.columns[[1] + list(range(5, Toronto2.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,East Toronto,0,Indian Restaurant,Sandwich Place,Gym,Brewery,Fast Food Restaurant,Bus Stop,Café,Coffee Shop,Park,Convenience Store
87,East Toronto,0,Fast Food Restaurant,Bar,Park,Pizza Place,Grocery Store,Italian Restaurant,Light Rail Station,Brewery,Burrito Place,Skate Park


In [125]:
Toronto2.loc[Toronto2['Cluster Labels'] == 1, Toronto2.columns[[1] + list(range(5, Toronto2.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,Downtown Toronto,1,Café,American Restaurant,Steakhouse,Coffee Shop,Pizza Place,Asian Restaurant,Breakfast Spot,Concert Hall,Hotel,Gastropub
60,Downtown Toronto,1,Café,Coffee Shop,Deli / Bodega,Restaurant,Hotel,Steakhouse,Bar,Gastropub,Japanese Restaurant,Salad Place
61,Downtown Toronto,1,Coffee Shop,Café,American Restaurant,Deli / Bodega,Restaurant,Hotel,Steakhouse,Gastropub,Gym,Beer Bar
70,Downtown Toronto,1,Café,Coffee Shop,Gastropub,Hotel,Deli / Bodega,American Restaurant,Gym,Bar,Restaurant,Concert Hall


In [126]:
Toronto2.loc[Toronto2['Cluster Labels'] == 2, Toronto2.columns[[1] + list(range(5, Toronto2.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,2,Pet Store,Playground,Music Venue,Garden,Dog Run,Dessert Shop,Diner,Discount Store,Dive Bar,Doner Restaurant


In [127]:
Toronto2.loc[Toronto2['Cluster Labels'] == 3, Toronto2.columns[[1] + list(range(5, Toronto2.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,3,Park,Bank,Trail,Grocery Store,Candy Store,Playground,Diner,Discount Store,Dive Bar,Dog Run


In [128]:
Toronto2.loc[Toronto2['Cluster Labels'] == 4, Toronto2.columns[[1] + list(range(5, Toronto2.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,4,Pub,Bar,Breakfast Spot,Sandwich Place,Café,Japanese Restaurant,Bookstore,Asian Restaurant,Gastropub,Bakery
41,East Toronto,4,Greek Restaurant,Coffee Shop,Pub,Bakery,Ice Cream Shop,Italian Restaurant,Café,Yoga Studio,Restaurant,Japanese Restaurant
54,Downtown Toronto,4,Café,Coffee Shop,Clothing Store,Theater,Cosmetics Shop,Ramen Restaurant,Italian Restaurant,Bookstore,Burger Joint,Restaurant
55,Downtown Toronto,4,Coffee Shop,Gastropub,Café,Restaurant,Hotel,Cocktail Bar,Beer Bar,Italian Restaurant,Farmers Market,Japanese Restaurant
56,Downtown Toronto,4,Café,Coffee Shop,Creperie,Farmers Market,Japanese Restaurant,Cheese Shop,Seafood Restaurant,Bakery,Cocktail Bar,Cosmetics Shop
59,Downtown Toronto,4,Hotel,Coffee Shop,Park,Brewery,Baseball Stadium,Café,Aquarium,Basketball Stadium,Skating Rink,Ice Cream Shop
65,Central Toronto,4,Café,Italian Restaurant,Pizza Place,History Museum,Jewish Restaurant,Tea Room,Pub,Mexican Restaurant,Mediterranean Restaurant,Vegetarian / Vegan Restaurant
66,Downtown Toronto,4,Café,Bakery,Bookstore,Vegetarian / Vegan Restaurant,Park,Japanese Restaurant,Pizza Place,Restaurant,Gaming Cafe,Dumpling Restaurant
67,Downtown Toronto,4,Café,Mexican Restaurant,Vegetarian / Vegan Restaurant,Burger Joint,Coffee Shop,Vietnamese Restaurant,Bakery,Bar,Caribbean Restaurant,Cocktail Bar
69,Downtown Toronto,4,Restaurant,Café,Bakery,Farmers Market,Cocktail Bar,Japanese Restaurant,Seafood Restaurant,Pub,Basketball Stadium,Italian Restaurant


In [129]:
Toronto2.loc[Toronto2['Cluster Labels'] == 5, Toronto2.columns[[1] + list(range(5, Toronto2.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,5,Gym / Fitness Center,Bookstore,Park,Café,Restaurant,Bus Line,Coffee Shop,General Entertainment,Design Studio,Dumpling Restaurant


In [130]:
Toronto2.loc[Toronto2['Cluster Labels'] == 6, Toronto2.columns[[1] + list(range(5, Toronto2.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,Central Toronto,6,Coffee Shop,Sporting Goods Shop,Italian Restaurant,Skating Rink,Café,Restaurant,Diner,Gym / Fitness Center,Salon / Barbershop,Rental Car Location
51,Downtown Toronto,6,Coffee Shop,Restaurant,Café,Park,Thai Restaurant,Gastropub,Diner,Italian Restaurant,Beer Store,Bakery
52,Downtown Toronto,6,Coffee Shop,Gastropub,Gay Bar,Japanese Restaurant,Men's Store,Restaurant,Pizza Place,Park,Mexican Restaurant,Pub
53,Downtown Toronto,6,Coffee Shop,Bakery,Café,Park,Thai Restaurant,Italian Restaurant,Breakfast Spot,Restaurant,Pub,Performing Arts Venue
75,Downtown Toronto,6,Korean Restaurant,Grocery Store,Coffee Shop,Café,Park,Pizza Place,Diner,Japanese Restaurant,Indian Restaurant,Cocktail Bar
76,West Toronto,6,Coffee Shop,Park,Gym,Pharmacy,Italian Restaurant,Supermarket,Bakery,Bar,Café,Portuguese Restaurant
78,West Toronto,6,Café,Coffee Shop,Bakery,Restaurant,Hotel,Gift Shop,Furniture / Home Store,Bar,Sandwich Place,Indian Restaurant
82,West Toronto,6,Bar,Italian Restaurant,Café,Grocery Store,Antique Shop,Flea Market,Bakery,Coffee Shop,Mexican Restaurant,Deli / Bodega
83,West Toronto,6,Bar,Sushi Restaurant,Gift Shop,Thai Restaurant,Eastern European Restaurant,Breakfast Spot,Bookstore,Café,Dog Run,Pizza Place


In [131]:
Toronto2.loc[Toronto2['Cluster Labels'] == 7, Toronto2.columns[[1] + list(range(5, Toronto2.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,7,Park,Grocery Store,Playground,Candy Store,Bank,Tennis Court,Thai Restaurant,Japanese Restaurant,Café,Sandwich Place


In [132]:
Toronto2.loc[Toronto2['Cluster Labels'] == 8, Toronto2.columns[[1] + list(range(5, Toronto2.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,East Toronto,8,Coffee Shop,Bakery,Café,Italian Restaurant,Bar,American Restaurant,Rock Club,Boutique,Restaurant,Comfort Food Restaurant
45,Central Toronto,8,Italian Restaurant,Café,Pizza Place,Coffee Shop,Park,Restaurant,Sushi Restaurant,Gym,Food & Drink Shop,Fast Food Restaurant
47,Central Toronto,8,Sushi Restaurant,Italian Restaurant,Café,Coffee Shop,Dessert Shop,Gym,Restaurant,Pizza Place,Indian Restaurant,Wings Joint
49,Central Toronto,8,Coffee Shop,Sushi Restaurant,Italian Restaurant,Spa,Pub,Café,German Restaurant,Restaurant,Pizza Place,Pharmacy
57,Downtown Toronto,8,Coffee Shop,Café,Italian Restaurant,Pizza Place,Japanese Restaurant,Bubble Tea Shop,Tea Room,Yoga Studio,Ramen Restaurant,Bar
64,Central Toronto,8,Italian Restaurant,Coffee Shop,Gym / Fitness Center,Asian Restaurant,Gastropub,Japanese Restaurant,Deli / Bodega,Park,Chinese Restaurant,Café
84,West Toronto,8,Coffee Shop,Café,Italian Restaurant,Bakery,Sushi Restaurant,Pizza Place,Pub,Gastropub,Falafel Restaurant,Park


In [133]:
Toronto2.loc[Toronto2['Cluster Labels'] == 9, Toronto2.columns[[1] + list(range(5, Toronto2.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Downtown Toronto,9,Harbor / Marina,Boat or Ferry,Airport Service,Airport Terminal,Sculpture Garden,Airport Lounge,Music Venue,Boutique,Rental Car Location,Pier
